In [2]:
import os
import glob

import numpy as np
import pandas as pd

#import cudf
#import cupy as cp
import nvtabular as nvt

from google.cloud import bigquery

import dask.dataframe as dd
from dask.distributed import Client
from dask.diagnostics import ProgressBar
client = Client(n_workers=4, threads_per_worker=4, memory_limit="1GB")
client

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./key/key.json"

distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-mu29yqlt', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-bisam48d', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-o_mcqj6f', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-_ud2xhte', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks/dask-worker-space/worker-3yb44t4p', purging
distributed.dis

In [2]:
df_2=pd.read_parquet("../parquet/transform_example_100.parquet")

In [12]:
df_2.isnull().sum()

article_count    0
article_id       0
Avg_price        0
description      0
detal_desc       0
dtype: int64

## Transformer

In [3]:
import texthero as hero
from texthero import preprocessing
from sentence_transformers import SentenceTransformer, util
import nmslib 
import time
import datetime

/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/.capestone/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


[SOurce](https://blog.berkgokden.com/is-it-possible-to-build-a-generic-recommender-system-with-deep-learning-and-nlp-ad9b2ed9560c)

In [2]:
from transformers import XLNetTokenizer, XLNetLMHeadModel

import torch

tokenizer = XLNetTokenizer.from_pretrained("xlnet-large-cased")

model = XLNetLMHeadModel.from_pretrained("xlnet-large-cased")


/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/.capestone/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is not available.  Training on CPU ...


In [7]:
input_ids = torch.tensor(tokenizer.encode(df_2["description"]))

In [12]:
# adds empty string to string array s if lenght is less than n
def pad_list(s, n):
    s = [string for string in s if string != ""]
    return [''] * (n - len(s)) + s 

# creates permutations of string array s with length n
def find_subsets(s, n):
    m = min(len(s), n)
    return list(itertools.permutations(s, m))

    # get a subsemple of permuations
def random_find_subsets(s, n, m):
    l = find_subsets(s, n)
    return sample(l, m)

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for seq in sequences:
        if len(seq) >= 3 and len(seq) <= 6:
            for inner_seq in random_find_subsets(seq, n_steps_in+n_steps_out, 3):
                padddes_seq = pad_list(inner_seq, n_steps_in+n_steps_out)
                seq_x, seq_y = padddes_seq[:n_steps_in], padddes_seq[-n_steps_out:]
                X.append(embed(seq_x))
                y.append(embed(seq_y))
    return array(X), array(y)

In [14]:
import hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

# load data and clean
df = pd.read_csv("data/data.csv", encoding= 'unicode_escape')
df = df.dropna()
df_descriptions = df[['Description']].dropna().drop_duplicates()
df_descriptions = df_descriptions[df_descriptions['Description'].str.len().gt(3)]

# these will be used for nearest neighboor search
descriptions = df_descriptions['Description'].tolist()
values = embed(descriptions)
np.save('data/descriptions.data', descriptions)
np.save('data/values.data', values)

# group transaction Descriptions by InvoiceNo
df_transactions = df[df['Description'].str.len().gt(3)].dropna().groupby('InvoiceNo')['Description'].agg({'size': len, 'set': lambda x: list(set(x))})
dataset = df_transactions['set'].tolist()

# split into 3 input steps and 2 output steps
n_steps_in, n_steps_out = 3, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)

np.save('data/X.data', X)
np.save('data/y.data', y)

DatasetHandlerError: A Hub dataset does not exist at the given path (https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3). Check the path provided or in case you want to create a new dataset, use hub.empty().

In [9]:
configuration = model.config

In [4]:
embeddings = XlnetEmbeddings.pretrained("xlnet_large_cased", "en") \
      .setInputCols("sentence", "token") \
      .setOutputCol("embeddings")

NameError: name 'XlnetEmbeddings' is not defined

In [14]:
df_2.describe().T

,count,mean,std,min,25%,50%,75%,max
article_count,100.0,10281.920000,5039.550478,6599.000000,7369.750000,8500.000000,11462.000000,42544.000000
Avg_price,100.0,0.022243,0.009564,0.005712,0.015939,0.023392,0.029616,0.048733


In [ ]:
embeddings = xlnet.encode(df_2['description'])
#embeddings2 =  xlnet.encode(df_2['detal_desc'])

In [10]:
df=pd.read_parquet("../parquet/transform_example_100.parquet")

In [6]:
from transformers import XLNetTokenizer, XLNetModel

tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')
model = XLNetModel.from_pretrained('xlnet-large-cased')


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
from tqdm import tqdm
from progress.bar import Bar

from pytorch_lightning.callbacks import TQDMProgressBar

  0%|          | 0/3 [09:25<?, ?it/s]


In [110]:

collect = {}

for i, j in  tqdm(zip(df["article_id"],df["description"])):
    inputs = tokenizer([j],  return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    collect[i]= last_hidden_states.detach().numpy()


100it [02:38,  1.58s/it]


In [26]:
df2 = df=pd.read_parquet("../parquet/transform_example_100.parquet")

In [24]:
def transform(j):
    inputs = tokenizer([j],  return_tensors="pt")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state
    
    return(last_hidden_states.detach().numpy())

  

In [25]:
  df["Xlnet"] = df["description"].apply(transform)

  0%|          | 0/1 [02:59<?, ?it/s]


In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:
def search_cloth2(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query =tokenizer([userQuery],  return_tensors="pt")
        ids, distances = distilbert_index2.knnQuery(query, k=12)
    

    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    matches = []    
    
    for i, j in zip(ids, distances):          
            matches.append({'article_id':df.article_id.values[i]
                        ,"article_count":df.article_count.values[i]
                        ,"price":df.Avg_price.values[i]
                        , 'description':df.description.values[i]
                        , 'detal_desc' : df.detal_desc.values[i]
                        , 'distance': j
                       })     
    return pd.DataFrame(matches)

In [34]:
df_3 = pd.read_csv("../data/XLnet_10000_all.csv").reset_index(drop=True)

In [38]:
df_3.columns

Index(['Unnamed: 0', 'article_count', 'article_id', 'Avg_price', 'description',
       'detail_desc', 'comb_desc', 'Xlnet', 'Xlnet_detail', 'Xlnet_comb'],
      dtype='object')

In [44]:
df_3.drop(columns='Unnamed: 0', axis=1)

,article_count,article_id,Avg_price,description,detail_desc,comb_desc,Xlnet,Xlnet_detail,Xlnet_comb
0,42544,706016001,0.032499,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Black ,Dark ,...",[[[-0.3804164 -0.8812267 -0.06792659 ... 2....,[[[-0.24108036 0.51591814 -0.5857612 ... 0....,[[[-0.04332412 -0.05329403 -1.1044266 ... 1....
1,29292,706016002,0.032449,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Light Blue ,L...",[[[-1.2474186 -0.10260493 -0.05328637 ... 1....,[[[-0.24108036 0.51591814 -0.5857612 ... 0....,[[[ 0.09275541 0.16334116 -0.9670675 ... 1....
2,21950,610776002,0.008091,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...",T-shirt in lightweight jersey with a rounded h...,"Tilly (1) ,Solid ,Black ,Dark ,Jersey Basic ,L...",[[[-0.29206443 -0.16364731 -1.3326098 ... -0....,[[[-0.47710276 2.1128855 -0.08665454 ... 1....,[[[-1.1898086 0.95437133 -1.5630703 ... 0....
3,19881,706016003,0.032700,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...",High-waisted jeans in washed superstretch deni...,"Jade HW Skinny Denim TRS ,Solid ,Dark Blue ,Me...",[[[-0.10801899 -1.1233265 0.34570506 ... 3....,[[[-0.24108036 0.51591814 -0.5857612 ... 0....,[[[ 0.8103858 0.12666857 -0.27916637 ... 1....
4,19462,562245001,0.026159,"Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...","5-pocket jeans in washed, superstretch denim w...","Luna skinny RW ,Solid ,Black ,Dark ,Trousers ,...",[[[-0.34007233 1.0228316 0.7145391 ... 0....,[[[-1.0142457 0.5663144 -0.3363339 ... 1....,[[[-0.12372264 0.6623686 -1.223984 ... 0....
...,...,...,...,...,...,...,...,...,...
9995,577,705974001,0.098150,"Mary dress ,Solid ,Dark Blue ,Dark ,Knitwear ,...","Knee-length dress in a sturdy, fine knit with ...","Mary dress ,Solid ,Dark Blue ,Dark ,Knitwear ,...",[[[ 1.1512377 1.0341833 1.0489019 ... -1....,[[[-1.2030926 0.89276254 -1.4924575 ... 0....,[[[ 1.0887921 1.7108539 -0.24746773 ... 0....
9996,577,853474005,0.035128,"Missy ,All over pattern ,Off White ,Dusty Ligh...","Voluminous, calf-length, off-the-shoulder dres...","Missy ,All over pattern ,Off White ,Dusty Ligh...",[[[ 1.2788656 0.5139149 -0.2915875 ... -3....,[[[-1.5335064 0.5242558 -1.2646139 ... 0....,[[[ 0.03329617 0.1860987 -0.52820575 ... 0....
9997,577,754758001,0.015908,"Annie Glove Touch ,Solid ,Black ,Dark ,Accesso...","Gloves in a soft, fine knit with a decorative ...","Annie Glove Touch ,Solid ,Black ,Dark ,Accesso...",[[[ 0.20464347 -1.7581607 0.22940135 ... 1....,[[[-2.1542487 1.7855182 -1.3432826 ... -0....,[[[ 0.12333435 -0.20989144 -1.470628 ... -0....
9998,577,732372002,0.021272,"NELLIE loose tee ,Solid ,Light Pink ,Dusty Lig...",Sports top in fast-drying functional fabric wi...,"NELLIE loose tee ,Solid ,Light Pink ,Dusty Lig...",[[[ 0.33143514 -0.62951034 0.0047082 ... -0....,[[[-0.1304922 -0.88622946 1.0710832 ... 2....,[[[-0.28871846 -0.22906666 0.47158793 ... -0....


In [29]:
!pwd

/Users/kaergell/Desktop/Neue Fische/Bootcamp/Capestone/fashion-recommender/notebooks


In [56]:
df["Xlnet"].

KeyError: 'XLNET'

In [57]:
XLNET_index = nmslib.init(method='hnsw', space='cosinesimil')
XLNET_index.addDataPointBatch(df_3["Xlnet"].astype(str))
XLNET_index.createIndex({'post': 2}, print_progress=True)

ValueError: could not convert string to float: '[[[-0.3804164  -0.8812267  -0.06792659 ...  2.3544111   0.4217048\n   -0.83486056]\n  [-0.31548518 -0.6912453  -0.45594075 ...  0.40584755 -0.0318665\n   -0.41349667]\n  [ 0.93621325  1.2570273   1.0034207  ...  2.0986671  -0.36189178\n   -1.9071089 ]\n  ...\n  [-1.5822608   1.4358767   1.5577245  ... -0.45630652 -0.6305022\n   -1.4402131 ]\n  [-1.5198551   1.8699065   0.3210218  ...  0.6321731  -1.4170187\n    0.07583806]\n  [-0.7321689   1.6952832   0.78677684 ...  0.59085864 -1.4562796\n    0.1934985 ]]]'